Basic 调参

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
 
path = "/content/gdrive/My Drive/Independent"
os.chdir(path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# 获取文本
import json

users_label = [] # 用户标签
users_posts = [] # 用户发言
index = 0    # 用户序号

# 读取文本
def read_text(file_path):
    global index

    text = open(file_path, 'r', encoding='UTF-8')
    line = text.readline()
    while line:
        if (len(line.strip()) != 0):
            setting = json.loads(line[1:-2])
            posts = setting['posts']
            label = setting['label']

            add_lable(label)
            add_posts(posts)

            index = index + 1
            line = text.readline()

        if (index == 1000):
            break

    return users_label, users_posts

# 记录用户标签
def add_lable(label):
    if (label == "depression"):
        users_label.append(1)
    else:
        users_label.append(0)

# 记录用户发言
def add_posts(posts):
    user_posts = ""
    for post in posts:
        user_posts = user_posts + " " + post[1]
    users_posts.append(user_posts)

In [ ]:
# 预处理
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# 词频
def count_vec(users_posts):
    count = CountVectorizer(stop_words='english')
    users_posts = count.fit_transform(users_posts)
    return users_posts

# TF-IDF
def tf_idf(users_posts):
    tfidf = TfidfVectorizer(stop_words='english')
    users_posts = tfidf.fit_transform(users_posts)
    return users_posts

In [ ]:
from sklearn.svm import SVC
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold

def support_vector(x_train, y_train):
    param_grid = {'C': [0.1, 1, 10, 100, 1000],
            'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
            'kernel': ['rbf'],
            'class_weight': ['balanced']}
    grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=3)
    grid.fit(x_train, y_train)

    print(grid.best_params_)

def xg_boost(x_train, y_train):
    param_grid = {'min_child_weight': [1, 5, 10],
            'gamma': [0.5, 1, 1.5, 2, 5],
            'subsample': [0.6, 0.8, 1.0],
            'colsample_bytree': [0.6, 0.8, 1.0],
            'max_depth': [3, 4, 5]}
    xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic', silent=True, nthread=1)

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1001)
    grid = RandomizedSearchCV(xgb, param_distributions=param_grid, n_iter=5, scoring='f1', n_jobs=4, verbose=3, random_state=1001)
    grid.fit(x_train, y_train)

    print(grid.best_params_)

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

# 获取用户标签和发言
file_path = "./docs/validation"
users_label, users_posts = read_text(file_path)

# 文本向量化
users_posts = count_vec(users_posts)
#users_posts = tf_idf(users_posts)

# 划分训练集和测试集
# 获取最佳参数
users_label = np.array(users_label)
ss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=1)
for train_idx, test_idx in ss.split(users_posts, users_label):
  x_train = users_posts[train_idx]
  y_train = users_label[train_idx]
  x_test = users_posts[test_idx]
  y_test = users_label[test_idx]

  #support_vector(x_train, y_train)
  #xg_boost(x_train, y_train)